In [2]:
import tabula 
import pandas as pd
from tika import parser
import os
import copy
import numpy as np

In [2]:
data2 = pd.read_csv("./dec/data/Declarations_third_table.csv", sep=';')
data2.head(2)

/home/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,NaN,D1221855500_00001_260716
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,NaN,D1221855500_00001_260716


In [3]:
dat = data2["destination"]
#Найменування,адреса, код згідно з ЄДРПОУ суб’єкта господарювання,
#якому передаються відходи;додатково серіята номер ліцензії у разі передачі небезпечних відходів

In [4]:
dat[0].split(',')

['38563155 ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»',
 ' 69002',
 ' м. Запоріжжя',
 ' вул. Червоногвардійська',
 ' буд. 40 Фактична адреса: 69000',
 ' м. Запоріжжя',
 ' вул. Теплична',
 ' 27. ',
 ' 08.01.2014']

In [5]:
dt = dat[:100]

In [94]:
def column_scatter(i):
    i = i['destination']
    edrpou = i.split()[0]
    if '.201' in i.split()[-1] or '.200' in i.split()[-1]:
        the_date = i.split()[-1]
        i = ' '.join(i.split()[1:-1])
    else:
        the_date = None
        i = ' '.join(i.split()[1:])
    name = i[:i.find(',')].strip()
    i = i[i.find(',') + 1:].strip()
    if 'Фактична адреса:' in i:
        address = i[:i.index('Фактична адреса:')].strip()
        fact_address = i[i.index('Фактична адреса:') + 16:].strip()
    else:
        address = i.strip()
        fact_address = None
#     print('edrpou', edrpou)
#     print('name', name)
#     print('adr', address)
#     print('fadr', fact_address)
#     print('date', the_date)
#     print('')
    return np.array(edrpou, name, address, fact_address, the_date)

In [102]:
def column_scatter(dt):
    edrpou_f, name_f, address_f, fact_address_f, the_date_f = [], [], [], [], []
    for i in dt:
        edrpou = i.split()[0]
        if '.201' in i.split()[-1] or '.200' in i.split()[-1]:
            the_date = i.split()[-1]
            i = ' '.join(i.split()[1:-1])
        else:
            the_date = None
            i = ' '.join(i.split()[1:])
        name = i[:i.find(',')].strip()
        i = i[i.find(',') + 1:].strip()
        if 'Фактична адреса:' in i:
            address = i[:i.index('Фактична адреса:')].strip()
            fact_address = i[i.index('Фактична адреса:') + 16:].strip()
        else:
            address = i.strip()
            fact_address = None
        edrpou_f.append(edrpou)
        name_f.append(name)
        address_f.append(address)
        fact_address_f.append(fact_address)
        the_date_f.append(the_date)
#     print('edrpou', edrpou)
#     print('name', name)
#     print('adr', address)
#     print('fadr', fact_address)
#     print('date', the_date)
#     print('')
    return edrpou_f, name_f, address_f, fact_address_f, the_date_f

In [103]:
data2['dest_edrpou'], data2['dest_name'], data2['dest_address'], data2['data_fact_address'], data2['dest_date'] = column_scatter(data2['destination'])

In [105]:
data2

,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num,dest_edrpou,dest_name,dest_address,data_fact_address,dest_date
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,NaN,D1221855500_00001_260716,38641065,"КП «Чисте місто» Жовтоводської міської ради""","52200,м. Жовті Води, вул. Петровського,56",None,None
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",None
4,5,Плівка чи оболонка на основі полімерів зіпсова...,1753.1.1.03,Відходи поліетилену,ІV,0,1.912,2.000,38563155 ТОВ «Науково- дослідний інститут «УКР...,1.912,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",None
5,6,Макулатура паперова та картонна,7710.3.1.01,Відходи макулатури,ІV,0,3.120,6.202,38563155 ТОВ «Науково- дослідний інститут «УКР...,3.120,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",None
6,7,Одяг зношений чи зіпсований,7710.3.1.13,Відходи спецодягу,ІV,0,0.104,0.195,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.104,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",None
7,1,Бій цегли,2640.2.9.02,Бій цегли,ІV,0,90.0,90.0,"26183742 філія ""Заставнівський райавтодор"", 59...",90.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",None,None
8,2,Шлак паливний,4010.2.8.01,Шлак паливний,ІV,0,69.0,69.0,"26183742 філія ""Заставнівський райавтодор"", 59...",69.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",None,None
9,1,Вичавки плодові та ягідні,1532.2.9.02,Вичавки плодові та ягідні,ІV,0,995.0,995.0,"36748453 КП ""Хотинблагоустрій"", Чернівецька об...",995.0,D1,NaN,D7325010100_00002_130716,36748453,"КП ""Хотинблагоустрій""","Чернівецька обл., м.Хотин, вул.Кобилянської, 15",None,None


In [107]:
# pd.read_csv("./dec/data/fulls/Declarations_third_splitted_table.csv", sep=';')

/home/roman/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,No,trash_name,code,alter_name,class,storage,value,value_prediction,destination,value_destination,operation,operation_desc,decl_num,dest_edrpou,dest_name,dest_address,data_fact_address,dest_date
0,1,"Масла та мастила моторні, трансмісійні нехлоро...",6000.2.8.09,Відпрацьовані мастила,ІІ,0,0.012,0.012,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.012,R9,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
1,2,"Матеріали обтиральні зіпсовані, відпрацьовані ...",7730.3.1.06,Промащене ганчір’я,ІІІ,0,0.100,0.211,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.100,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27. ,",08.01.2014
2,3,"Відходи комунальні (міські) змішані, у т. ч. с...",7720.3.1.01,Побутове сміття,ІV,0,18.000,84.18,38641065 КП «Чисте місто» Жовтоводської місько...,18.000,D1,NaN,D1221855500_00001_260716,38641065,"КП «Чисте місто» Жовтоводської міської ради""","52200,м. Жовті Води, вул. Петровського,56",NaN,NaN
3,4,Підметини та пух хутра штучного,1830.2.1.06,Підметини з цехів,ІV,0,35.200,40.5,38563155 ТОВ «Науково- дослідний інститут «УКР...,35.200,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
4,5,Плівка чи оболонка на основі полімерів зіпсова...,1753.1.1.03,Відходи поліетилену,ІV,0,1.912,2.000,38563155 ТОВ «Науково- дослідний інститут «УКР...,1.912,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
5,6,Макулатура паперова та картонна,7710.3.1.01,Відходи макулатури,ІV,0,3.120,6.202,38563155 ТОВ «Науково- дослідний інститут «УКР...,3.120,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
6,7,Одяг зношений чи зіпсований,7710.3.1.13,Відходи спецодягу,ІV,0,0.104,0.195,38563155 ТОВ «Науково- дослідний інститут «УКР...,0.104,R5,NaN,D1221855500_00001_260716,38563155,ТОВ «Науково- дослідний інститут «УКРЕКОПРОЕКТ»,"69002, м. Запоріжжя, вул. Червоногвардійська, ...","69000, м. Запоріжжя, вул. Теплична, 27.",NaN
7,1,Бій цегли,2640.2.9.02,Бій цегли,ІV,0,90.0,90.0,"26183742 філія ""Заставнівський райавтодор"", 59...",90.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",NaN,NaN
8,2,Шлак паливний,4010.2.8.01,Шлак паливний,ІV,0,69.0,69.0,"26183742 філія ""Заставнівський райавтодор"", 59...",69.0,D1,NaN,D7321586001_00001_100616,26183742,"філія ""Заставнівський райавтодор""","59400, Чернівецька обл., м.Заставна, вул.Бажан...",NaN,NaN
9,1,Вичавки плодові та ягідні,1532.2.9.02,Вичавки плодові та ягідні,ІV,0,995.0,995.0,"36748453 КП ""Хотинблагоустрій"", Чернівецька об...",995.0,D1,NaN,D7325010100_00002_130716,36748453,"КП ""Хотинблагоустрій""","Чернівецька обл., м.Хотин, вул.Кобилянської, 15",NaN,NaN


зробити спліт в наборах "Перелік установ, організацій та закладів, які здійснюють розробку документів, що обгрунтовують обсяги викидів для підприємств, установ, організацій та громадян-суб'єктів" в стовпці "Повне найменування установи, організації, закладу, код ЄДРПОУ", "Ліцензійний реєстр (поводження з небезпечними відходами)" Код це ключ, його потрібно виокремити; 

In [38]:
data = pd.read_excel("Перелік установ, організацій та закладів, які здійснюють розробку документів, що обгрунтовують обсяги викидів для підпри.xlsx", skiprows=1)

In [39]:
data.head(5)

,"Повне найменування установи, організації, закладу, код ЄДРПОУ","Місцезнаходження, телефон","Назва області, код за КОАТУУ",Прізвище та ініціали керівника,Дата внесення організації до переліку,"Термін, до якого організацію внесено до переліку",Дата виключення організації з переліку
0,"Обласне комунальне підприємство ""Донецьктеплок...","83086, Донецька обл., м.Донецьк, вул. Донецька...","14-Донецька область, 1410136300",Поляков Р.О.,"20.02.2014, №5/2-12/2460-14",20.02.2019,NaN
1,"ТОВ Сільськогосподарське підприємство ""Нібуло...","54002, Миколаївська обл., м.Миколаїв, Каботажн...","48-Миколаївська область, 4810136300",Вадатурський О.О.,"20.02.2014, №5/2-12/2462-14",20.02.2019,NaN
2,Обласне комунальне спеціалізоване підприємство...,"61022, Харківська обл., м.Харків, майдан Свобо...","63-Харківська область, 6310136300",Шпаркий С.А.,"18.02.2014, №5/2-12/2399-14",18.02.2019,NaN
3,"ПАТ ""По газопостачанню та газифікації ""Донецьк...","83050, Донецька обл., м.Донецьк, вул.Рози Люкс...","14-Донецька область, 1410136300",Масс М.С.,"20.02.2014, №5/2-12/2461-14",20.02.2019,NaN
4,"""Експертно-методична лабораторія"" підприємець ...","43026, Волинська обл., м.Луцьк, вул. Кравчука,...","07-Волинська область, 0710100000",Навроцький О.М.,"20.02.2014, №5/2-12/2463-14",20.02.2019,NaN


In [40]:
data['код ЄДРПОУ'] = data['Повне найменування установи, організації, закладу, код ЄДРПОУ'].apply(lambda x: x[x.rindex(',') + 1:].strip())
data['Повне найменування установи, організації, закладу'] = data['Повне найменування установи, організації, закладу, код ЄДРПОУ'].apply(lambda x: x[:x.rindex(',')].strip())
del data['Повне найменування установи, організації, закладу, код ЄДРПОУ']
del data['Дата виключення організації з переліку']

In [41]:
data

,"Місцезнаходження, телефон","Назва області, код за КОАТУУ",Прізвище та ініціали керівника,Дата внесення організації до переліку,"Термін, до якого організацію внесено до переліку",код ЄДРПОУ,"Повне найменування установи, організації, закладу"
0,"83086, Донецька обл., м.Донецьк, вул. Донецька...","14-Донецька область, 1410136300",Поляков Р.О.,"20.02.2014, №5/2-12/2460-14",20.02.2019,03337119,"Обласне комунальне підприємство ""Донецьктеплок..."
1,"54002, Миколаївська обл., м.Миколаїв, Каботажн...","48-Миколаївська область, 4810136300",Вадатурський О.О.,"20.02.2014, №5/2-12/2462-14",20.02.2019,14291113,"ТОВ Сільськогосподарське підприємство ""Нібулон"""
2,"61022, Харківська обл., м.Харків, майдан Свобо...","63-Харківська область, 6310136300",Шпаркий С.А.,"18.02.2014, №5/2-12/2399-14",18.02.2019,31519314,Обласне комунальне спеціалізоване підприємство...
3,"83050, Донецька обл., м.Донецьк, вул.Рози Люкс...","14-Донецька область, 1410136300",Масс М.С.,"20.02.2014, №5/2-12/2461-14",20.02.2019,03361075,"ПАТ ""По газопостачанню та газифікації ""Донецьк..."
4,"43026, Волинська обл., м.Луцьк, вул. Кравчука,...","07-Волинська область, 0710100000",Навроцький О.М.,"20.02.2014, №5/2-12/2463-14",20.02.2019,1997714358,"""Експертно-методична лабораторія"" підприємець ..."
5,"03055, м.Київ, вул. Борщагівська, 117, оф.247,...","80-м.Київ, 8038600000",Сябро М.М.,"25.02.2014, №5/3-12/2563-14",25.02.2019,38250203,"ТОВ ""Геоекспертресурс"""
6,"03087, м.Київ, вул. Іскрівська, 1, к.23, тел.(...","80-Київ, 8038900000",Салан С.С.,"27.02.2014, №5/3-12/2679-14",27.02.2019,38716120,"ТОВ ""МЕХЕКО"""
7,"61022, Харківська обл., м.Харків, Майдан Свобо...","63-Харківська область, 6310136300",Кучеров К.І.,"26.02.2014, №5/3-12/2623-14",26.02.2019,21254147,Асоціація Харківського міжгалузевого центру ек...
8,"18000, Черкаська обл., м.Черкаси, вул. Байди В...","71-Черкаська область, 7110136700",Романчук В.А.,"28.02.2014, №5/3-12/2729-14",28.02.2019,38595131,"ПП ""Центр утилізації"""
9,"43005, Волинська обл., м.Луцьк, вул.Гулака-Арт...","07-Волинська область, 0710100000",Бакараєв О.А.,"11.03.2014, №5/3-12/2918-14",11.03.2019,38474280,"ТОВ ""Волиньекопромпроект"""


In [43]:
data.to_csv("Перелік установ, організацій та закладів, які здійснюють розробку документів, що обгрунтовують обсяги викидів для підпри.csv", sep=';', index=False)

In [108]:
pd.read_csv('Перелік установ, організацій та закладів, які здійснюють розробку документів, що обгрунтовують обсяги викидів для підпри.csv', sep=';')

,"Місцезнаходження, телефон","Назва області, код за КОАТУУ",Прізвище та ініціали керівника,Дата внесення організації до переліку,"Термін, до якого організацію внесено до переліку",код ЄДРПОУ,"Повне найменування установи, організації, закладу"
0,"83086, Донецька обл., м.Донецьк, вул. Донецька...","14-Донецька область, 1410136300",Поляков Р.О.,"20.02.2014, №5/2-12/2460-14",20.02.2019,3337119,"Обласне комунальне підприємство ""Донецьктеплок..."
1,"54002, Миколаївська обл., м.Миколаїв, Каботажн...","48-Миколаївська область, 4810136300",Вадатурський О.О.,"20.02.2014, №5/2-12/2462-14",20.02.2019,14291113,"ТОВ Сільськогосподарське підприємство ""Нібулон"""
2,"61022, Харківська обл., м.Харків, майдан Свобо...","63-Харківська область, 6310136300",Шпаркий С.А.,"18.02.2014, №5/2-12/2399-14",18.02.2019,31519314,Обласне комунальне спеціалізоване підприємство...
3,"83050, Донецька обл., м.Донецьк, вул.Рози Люкс...","14-Донецька область, 1410136300",Масс М.С.,"20.02.2014, №5/2-12/2461-14",20.02.2019,3361075,"ПАТ ""По газопостачанню та газифікації ""Донецьк..."
4,"43026, Волинська обл., м.Луцьк, вул. Кравчука,...","07-Волинська область, 0710100000",Навроцький О.М.,"20.02.2014, №5/2-12/2463-14",20.02.2019,1997714358,"""Експертно-методична лабораторія"" підприємець ..."
5,"03055, м.Київ, вул. Борщагівська, 117, оф.247,...","80-м.Київ, 8038600000",Сябро М.М.,"25.02.2014, №5/3-12/2563-14",25.02.2019,38250203,"ТОВ ""Геоекспертресурс"""
6,"03087, м.Київ, вул. Іскрівська, 1, к.23, тел.(...","80-Київ, 8038900000",Салан С.С.,"27.02.2014, №5/3-12/2679-14",27.02.2019,38716120,"ТОВ ""МЕХЕКО"""
7,"61022, Харківська обл., м.Харків, Майдан Свобо...","63-Харківська область, 6310136300",Кучеров К.І.,"26.02.2014, №5/3-12/2623-14",26.02.2019,21254147,Асоціація Харківського міжгалузевого центру ек...
8,"18000, Черкаська обл., м.Черкаси, вул. Байди В...","71-Черкаська область, 7110136700",Романчук В.А.,"28.02.2014, №5/3-12/2729-14",28.02.2019,38595131,"ПП ""Центр утилізації"""
9,"43005, Волинська обл., м.Луцьк, вул.Гулака-Арт...","07-Волинська область, 0710100000",Бакараєв О.А.,"11.03.2014, №5/3-12/2918-14",11.03.2019,38474280,"ТОВ ""Волиньекопромпроект"""
